# Решение тестового задания

## Задача: Восстановить данные, привести данные в удобочитаемый вид.

## Дано: Excel-таблица "Тестовое задание для стажировки по аналитике_ Датасет" с выгрузкой данных.

## Решение:

### Исходя из выгрузки, обнаруживаем, что дано две таблицы. Для удобства работы с данными разделим эти таблицы по разным листам и приведем их к началу таблицы.

### Работаем с выгрузкой из реестра. Видим, что все записи имеют стандартный вид "пациент [ID] оплатил [loss] от [date]". Напишем небольшой код для разделения этих данных и приведения их к удобочитаемой таблице.

In [183]:
# Загружаем библиотеки
import pandas as pd
import xlrd

# Приводим к списку, сразу его разбиваем

wb = xlrd.open_workbook(r'C:\Users\Ильнур\Desktop\Тестовое задание для стажировки по аналитике_ Датасет.xlsx')
sheet = wb.sheet_by_index(0)
sheet.cell_value(0, 0)
row_list = []
for i in range(sheet.nrows):
    try:
        key = sheet.cell_value(i, 0)
        row_list.append(key.split(' '))
    except (ValueError, IndexError):
        continue
row_list

# Создаем датафрейм, убираем мусорные столбцы и выводимм первые 10 строк

df = pd.DataFrame(row_list)
df = df.drop([0, 2, 4], 1)
df = df.rename(columns={1:'ID', 3:'Loss', 5:'Date'})
df.head(10)

,ID,Loss,Date
0,21,3468.72,7.1.2020
1,22,9264.84,2.1.2020
2,23,9593.12,31.1.2020
3,24,7798.25,30.1.2020
4,25,4425.30,24.1.2020
5,26,4019.86,10.26.2020
6,27,6508.98,11.9.2020
7,28,4447.99,1.29.2020
8,29,4111.23,22.1.2020
9,30,7877.01,26.2.2020


In [184]:
# Загрузим также вторую таблицу, используем её позже

df2 = pd.read_excel(r'C:\Users\Ильнур\Desktop\Тестовое задание для стажировки по аналитике_ Датасет.xlsx', sheet_name='Лист1')
df2

,Месяц,Сумма
0,январь,485487.84
1,февраль,570192.91
2,март,698723.40
3,апрель,444834.87
4,май,574488.89
5,июнь,579810.45


### С этими данными уже можно работать. Рассмотрим поближе наш датафрейм, а именно, обратим внимение на типы данных

In [185]:
df.dtypes

ID      object
Loss    object
Date    object
dtype: object

### Наблюдаем, что столбец Loss - object. Для ведения счёта приведем его в float: 

In [186]:
df["Loss"] = pd.to_numeric(df["Loss"])
df.dtypes

ID       object
Loss    float64
Date     object
dtype: object

### Эти данные готовы к выгрузке. Однако при детальном рассмотрении столбца Date, мы видим аномалии в некоторых значениях, например:

In [187]:
df.iloc[4:6]

,ID,Loss,Date
4,25,4425.30,24.1.2020
5,26,4019.86,10.26.2020


### Таким образом, либо нарушен порядок присвоения дат и месяцев (где-то DD.MM.YYYY, а где-то MM.DD.YYYY), либо дни и месяцы просто заменены случайными числами от 1 до 31.

## Теория: Ввиду того, что в нашей выгрузке ID пациента идут последовательно, а по колонке Date на второй позиции мы видим наиболее часто встречающуюся цифру предполагаемого месяца, есть основания полагать, что выгрузка сортирована по дате.

### Проверим эту теорию. Для этого взглянем на выборку и выберем предполагаемые граничные значения каждого месяца, просуммируем по каждому предположенному месяцу и сравним со второй данной нам таблицей.

In [188]:
print('Январь -', df.Loss[0:90].sum())
print('Февраль -', df.Loss[90:200].sum())
print('Март -', df.Loss[200:330].sum())
print('Апрель -', df.Loss[330:420].sum())
print('Май -', df.Loss[420:530].sum())
print('Июнь -', df.Loss[530:].sum())

Январь - 485487.8399999999
Февраль - 570192.9099999999
Март - 698723.4
Апрель - 444834.87
Май - 574488.89
Июнь - 579810.4500000001


In [189]:
df2

,Месяц,Сумма
0,январь,485487.84
1,февраль,570192.91
2,март,698723.40
3,апрель,444834.87
4,май,574488.89
5,июнь,579810.45


### Визуально подобранные границы месяцев на 100% совпадают с данной нам выгрузкой по месяцам. 
### Таким образом, наша теория подтвердилась. Индекс последней оплаты в первом месяце - 89 (нумерация в Python идет с 0), во втором - 199 и т.д.  На основании этого приводим таблицу согласно запросу Отдела реестров:

In [190]:
# Заменяем столбец даты на номер месяца, переименовыаем согласно примеру ответа
df = df.rename(columns={'ID':'patiend_id', 'Date':'report_month'})
df = df[['patiend_id', 'report_month', 'Loss']]


In [191]:
df.loc[df.index[0:90], 'report_month'] = 1
df.loc[df.index[90:200], 'report_month'] = 2
df.loc[df.index[200:330], 'report_month'] = 3
df.loc[df.index[330:420], 'report_month'] = 4
df.loc[df.index[420:530], 'report_month'] = 5
df.loc[df.index[530:], 'report_month'] = 6
df

,patiend_id,report_month,Loss
0,21,1,3468.72
1,22,1,9264.84
2,23,1,9593.12
3,24,1,7798.25
4,25,1,4425.30
...,...,...,...
655,606,6,6548.01
656,607,6,6597.90
657,608,6,2526.00
658,609,6,8097.32


In [192]:
# Экспорт файла

df.to_csv(r'C:\Users\Ильнур\Desktop\Каримуллин Ильнур.csv', index=False)

### P.S. Решение, приведенное выше, имеет место быть благодаря небольшой выборке. Однако, если в следующий раз Отдел реестров пришлет нам такое же задание, но уже с выборкой за пять лет, то потребуется гораздо больше времени для определения границ месяца. В этом случае рациональнее будет использовать кумулятивную сумму по значению loss. 

### Применение кумулятивной суммы может выглядеть так:

In [193]:
# Загружаем нужную библиотеку
import numpy as np

# Обрабатываем поэлементно нужные столбцы и записываем их в отдельные переменные

sum_by_payment = [int(i) for i in np.cumsum(df['Loss']).values]
sum_by_month = [int(i) for i in np.cumsum(df2['Сумма']).values]

# Пробегаемся целевым списком по значениям, выводим индексы совпадающих элементов
count = 0
for i in sum_by_month:
    count += 1
    if i in sum_by_payment:
        print(f'Крайний индекс месяца {count} -', sum_by_payment.index(i))

Крайний индекс месяца 1 - 89
Крайний индекс месяца 2 - 199
Крайний индекс месяца 3 - 329
Крайний индекс месяца 4 - 419
Крайний индекс месяца 5 - 529
Крайний индекс месяца 6 - 659
